# AutoGen with MCP Built-in Echo Server Tutorial

This notebook demonstrates how to use AutoGen with the Model Context Protocol (MCP) using the built-in Python echo server.

## What is MCP?

The Model Context Protocol (MCP) allows AI agents to access external tools and data sources in a standardized way. Think of it as a universal connector that lets your AI agent use various tools like file systems, databases, APIs, and more.

## Prerequisites

First, let's install the required packages:

In [1]:
# Install dependencies
!pip install --quiet -U "autogen-agentchat>=0.7" "autogen-ext[openai,mcp]>=0.7" "mcp>=1.0.0"

# IMPORTANT: See: https://github.com/microsoft/autogen/issues/6906
!pip install --quiet --force-reinstall "openai==1.80"

In [2]:
%%writefile mcp_echo_tool.py
"""
FastMCP Echo Server with Logging
"""
import logging
from mcp.server.fastmcp import FastMCP

logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Create server
mcp = FastMCP("Echo Server")

@mcp.tool()
def echo(text: str) -> str:
    """Echo the input text"""
    logging.info(f"ECHO TOOL CALLED with text: '{text}'")
    return text

if __name__ == "__main__":
    logging.info("Starting Echo Server...")
    mcp.run(transport="streamable-http")

Overwriting mcp_echo_tool.py


```bash
sudo apt-get update
sudo apt-get install npm
pip install uv mcp[cli]
python mcp_echo_tool.py
```

## Step 1: Verify MCP Echo Server

Let's first verify that the built-in echo server is available:

## Step 2: Set Up Your OpenAI API Key

## Step 3: Create a Simple AutoGen Agent with MCP Echo Server

In [3]:
# AutoGen with MCP Echo Server
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.mcp import McpWorkbench, StreamableHttpServerParams, StreamableHttpMcpToolAdapter
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

async def run_echo_example():
    """Run the AutoGen MCP echo server example"""
    
    # Define the MCP echo server
    server_params = StreamableHttpServerParams(
        url="http://127.0.0.1:8000/mcp"
    )
    adapter = await StreamableHttpMcpToolAdapter.from_server_params(server_params, "echo")
    
    model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
    
    # Create assistant agent with MCP tools
    agent = AssistantAgent(
        name="echo_assistant",
        model_client=model_client,
        tools=[adapter],
        system_message="You are a helpful assistant that can use the echo tool to repeat messages back to users."
    )
    
    # Test the echo functionality
    tasks = [
        "Use the echo tool to say 'Hello, MCP World!'",
        "Echo back 'AutoGen + MCP is awesome!'",
        "Can you echo 'This is working perfectly'?"
    ]
    
    for i, task in enumerate(tasks, 1):
        print(f"\n🔄 Task {i}: {task}")
        print("-" * 40)

        await Console(
            agent.run_stream(task=task, cancellation_token=CancellationToken())
        )
        
    await model_client.close()

await run_echo_example()


🔄 Task 1: Use the echo tool to say 'Hello, MCP World!'
----------------------------------------
---------- TextMessage (user) ----------
Use the echo tool to say 'Hello, MCP World!'
---------- ToolCallRequestEvent (echo_assistant) ----------
[FunctionCall(id='call_vLmJpewMS6gEj6ComWMfWM3S', arguments='{"text":"Hello, MCP World!"}', name='echo')]
---------- ToolCallExecutionEvent (echo_assistant) ----------
[FunctionExecutionResult(content='[{"type": "text", "text": "Hello, MCP World!", "annotations": null}]', name='echo', call_id='call_vLmJpewMS6gEj6ComWMfWM3S', is_error=False)]
---------- ToolCallSummaryMessage (echo_assistant) ----------
[{"type": "text", "text": "Hello, MCP World!", "annotations": null}]

🔄 Task 2: Echo back 'AutoGen + MCP is awesome!'
----------------------------------------
---------- TextMessage (user) ----------
Echo back 'AutoGen + MCP is awesome!'
---------- ToolCallRequestEvent (echo_assistant) ----------
[FunctionCall(id='call_MJlpK5TGR5AmRx01fvMirKOq', arg

## Step 4: Understanding What Happened

🔍 What just happened?

1. **MCP Server Setup**: We created a StdioServerParams that points to the built-in echo server.
   
2. **Workbench Connection**: McpWorkbench manages the connection between AutoGen and the MCP server.
   
3. **Agent Creation**: Our AssistantAgent was configured to use the workbench, giving it access to the server's tools.
   
4. **Tool Usage**: The agent automatically discovered and used the 'echo' tool when prompted.

The echo server provides a simple tool that takes a message and echoes it back.
This demonstrates the basic MCP workflow: Agent → MCP Client → MCP Server → Tool → Response

## Next Steps


🎉 Congratulations! You've successfully:

✅ Set up AutoGen with MCP
✅ Connected to a built-in Python MCP server  
✅ Created an agent that can use MCP tools
✅ Tested the echo functionality

🚀 Next Steps:

1. **Try Other Built-in Servers**:
   - Memory server: python -m mcp_sdk.mcp_client.examples.memory
   - SQLite server: python -m mcp_sdk.mcp_client.examples.sqlite

2. **Create Custom MCP Servers**:
   - Use FastMCP for simpler Python servers
   - Implement your own tools and capabilities

3. **Explore Community Servers**:
   - File system operations
   - Web scraping tools  
   - Database integrations
   - API connectors

4. **Build Real Applications**:
   - Data analysis agents
   - File processing workflows
   - Multi-agent systems with shared tools

📚 Resources:
- MCP Documentation: https://modelcontextprotocol.io
- AutoGen Documentation: https://microsoft.github.io/autogen
- Python MCP SDK: https://github.com/modelcontextprotocol/python-sdk

## Summary

This tutorial showed you how to integrate AutoGen with MCP using the built-in echo server. The echo server is perfect for learning because it's simple but demonstrates all the key MCP concepts:

- **Server Connection**: How AutoGen connects to MCP servers
- **Tool Discovery**: How agents find available tools
- **Tool Usage**: How agents invoke MCP tools automatically
- **Response Handling**: How results flow back to the agent

The same patterns work with any MCP server - file systems, databases, APIs, or custom tools you build yourself!